# はじめに
このコンペでは3カ国にある2つの店舗(Mart, Rama)で3つの商品(mug, hat, sticker)の1年分の売上を予測する。

売れ行きには週末や祝日の効果、季節など、多くの要因が作用している。

このデータセットは小さいので多くのモデリングアプローチを試すことができる。

In [ ]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(CSS))

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

plt.rcParams['font.family'] = 'IPAexGothic'


train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

## 探索的EDA

In [ ]:
# import pandas_profiling as pdp

# pdp.ProfileReport(train)

## カラム(6個)
* row_id : ID
* date : 日付(YYYY-MM-dd)
* country : 国
    * フィンランド、ノルウェイ、スウェーデン
* store : 架空の店
    * KaggleMart、KaggleRama
* product : 製品
    * Kaggle Mug(マグカップ)
    * Kaggle Hat(帽子)
    * Kaggle Sticker(ステッカー)
* num_sold : 売れた個数

In [ ]:
train

In [ ]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

country, store, productの3カラムの総計

In [ ]:
train.describe(include='O')

In [ ]:
for col in ['country', 'store', 'product']:
    print(col, train[col].unique())

訓練用データセットは2015年~2018年、テスト用データセットは2019年

In [ ]:
print("trainデータの期間 : ", train['date'].min(), train['date'].max())
print("testデータの期間 : ", test['date'].min(), test['date'].max())

In [ ]:
for col in ['country', 'store', 'product']:
    display(pd.DataFrame(train[col].value_counts()))

In [ ]:
for col in ['country', 'store', 'product']:
    display(pd.DataFrame(test[col].value_counts()))

# 前処理とデータ加工

* 年月日によるピボットテーブルを作成

In [ ]:
train_date = train.set_index('date').pivot(columns=['country', 'store', 'product'], values='num_sold')
train_date.head()

* 月ごとのピボットテーブルを作成
    * unstack()で行方向から列方向にピボット

In [ ]:
train_month = train.set_index('date').groupby([pd.Grouper(freq='M'), 'country', 'store', 'product'])['num_sold'].mean().unstack([1, 2, 3])
train_month_country = train.set_index('date').groupby([pd.Grouper(freq='M'), 'country'])['num_sold'].mean().unstack()
train_month.head()

月別にグルーピングすることで全体の傾向を把握しやすくなる。年末年初にピークを迎え、その前後で減少する傾向が見て取れる。

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
train_monthly = train.set_index('date').groupby([pd.Grouper(freq='M')])[['num_sold']].mean()

sns.lineplot(x='date', y='num_sold', data=train, ax=ax, label='daily')
sns.lineplot(x='date', y='num_sold', data=train_monthly, ax=ax, label='monthly_mean', color='black')
ax.set_title('Monthly Trend', fontsize=20, fontweight='bold', loc='left', y=1.03)
ax.grid(alpha=0.5)
ax.legend()
plt.show()

月別、国別にグルーピングして売上個数の推移を見る

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
train_month_country = train.set_index('date').groupby([pd.Grouper(freq='M'), 'country'])[['num_sold']].mean()
sns.lineplot(x='date', y='num_sold', hue='country', data=train_month_country, ax=ax)

ax.set_ylabel('num_sold')
ax.set_title('Monthly Trend by Country', fontsize=15, fontweight='bold', loc='left', y=1.03)
ax.grid(alpha=0.5)
ax.legend()
plt.show()

# 曜日
商品の売れ行きなどの時系列データは週末と平日で分布が異なることが多い。特徴量として曜日を使うことは有効な場合が多い。

In [ ]:
train['dayofweek'] = train['date'].dt.dayofweek
test['dayofweek'] = test['date'].dt.dayofweek

# 曜日別の売れ行きを確認する
* 0: mon
* 1: tue
* 2: wed
* 3: thu
* 4: fri
* 5: sat
* 6: sun

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
train_dayofweek = train.set_index('date').groupby([pd.Grouper(freq='M'), 'dayofweek'])[['num_sold']].mean()

sns.lineplot(x='date', y='num_sold', hue='dayofweek', data=train_dayofweek, ax=ax, palette='deep')
ax.set_title('Trend day of week', fontsize=15, fontweight='bold', loc='left')
ax.grid(alpha=0.5)
plt.show()


休日と平日で売れ行きに差があるので比較する

In [ ]:
train['weekend'] = train['dayofweek'].apply(lambda x: x>=5)
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
train_weekend = train.set_index('date').groupby([pd.Grouper(freq='M'), 'weekend'])[['num_sold']].mean()

sns.lineplot(x='date', y='num_sold', hue='weekend', data=train_weekend, ax=ax)
ax.set_title('Weekend vs Weekday Trend Comparison', fontsize=15, fontweight='bold', loc='left')
ax.grid(alpha=0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
country_dayofweek = pd.pivot_table(train, index='country', columns='dayofweek', values='num_sold', aggfunc=np.mean)
country_dayofweek = pd.DataFrame(country_dayofweek.divide(country_dayofweek.sum(axis=1), axis=0).unstack()).reset_index(level=[0,1])
country_dayofweek.rename(columns={0:'num_sold'}, inplace=True)
# country_dayofweek.reset_index(level=[0,1])
sns.barplot(x='dayofweek', y='num_sold', hue='country',data=country_dayofweek, ax=ax)
ax.grid(axis='y',alpha=0.5, )
ax.set_xticklabels(['MON', 'TUE', 'WED','THU','FRI','SAT','SUN'])
ax.set_title('Percentage by day of the week by country', fontsize=15, fontweight='bold', loc='left')
plt.show()

金曜日も他の平日に比べて高い割合になっている

* 国別製品比率
* 国ごとの製品率はあまり大差はない

In [ ]:
country_product = pd.pivot_table(train, index='country', columns='product', values='num_sold', aggfunc=np.mean)
country_product.divide(country_product.sum(axis=1), axis=0)

曜日ごとの製品率はあまり大差はない

In [ ]:
country_product_dayofweek = pd.pivot_table(train, index='dayofweek', columns='product', values='num_sold', aggfunc=np.mean)
country_product_dayofweek.divide(country_product_dayofweek.sum(axis=1), axis=0)

# Bar Chart Race

In [ ]:
!pip install -qqq bar_chart_race

In [ ]:
import bar_chart_race as bcr


bcr.bar_chart_race(df=train_month,
                  n_bars=9,
                  period_length=800,
                  filename=None)